In [1]:
# Clone repo and install dependencies

!git clone https://github.com/google/automl.git
!pip install tensorflow_addons

Cloning into 'automl'...
remote: Enumerating objects: 3996, done.
remote: Counting objects: 100% (383/383), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 3996 (delta 226), reused 297 (delta 184), pack-reused 3613
Receiving objects: 100% (3996/3996), 25.16 MiB | 19.46 MiB/s, done.
Resolving deltas: 100% (2978/2978), done.
     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [2]:
%cd automl/efficientnetv2/

/content/automl/efficientnetv2


In [3]:
import os
import numpy as np
import tensorflow as tf

from effnetv2_model import EffNetV2Model
import preprocessing


In [4]:
# Load data

image_file = 'panda.jpg'
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O {image_file}

--2021-09-10 08:57:01--  https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.153.240, 2620:0:860:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.153.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: ‘panda.jpg’

panda.jpg           100%[===================>] 113.35K  --.-KB/s    in 0.07s   

2021-09-10 08:57:01 (1.62 MB/s) - ‘panda.jpg’ saved [116068/116068]



In [ ]:
MODELS = [
    "efficientnetv2-b0",
    "efficientnetv2-b1",
    "efficientnetv2-b2",
    "efficientnetv2-b3",
    "efficientnetv2-s",
    "efficientnetv2-m",
    "efficientnetv2-l",
]

In [ ]:
# Run for Imagenet-1k pretrained logits:

for name in MODELS:
    tf.keras.backend.clear_session()
  
    !curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{name}.tgz | tar xzf -

    model = EffNetV2Model(name)

    # Initialize variables:
    _ = model(tf.ones([1, model.cfg.eval.isize, model.cfg.eval.isize, 3]), training=False)
    
    # Load weights:
    ckpt_path = os.path.join(os.getcwd(), name)
    if tf.io.gfile.isdir(ckpt_path):
        ckpt_path = tf.train.latest_checkpoint(ckpt_path)
    model.load_weights(ckpt_path)

    print(f"Model variant: {name}")
    print(f"Train image size: {model.cfg.train.isize}")
    print(f"Eval image size: {model.cfg.eval.isize}")
    print()

    image = tf.image.decode_jpeg(tf.io.read_file(image_file))
    input_tensor = preprocessing.preprocess_image(
        image,
        image_size=model.cfg.eval.isize,
        is_training=False
    )
    input_tensor = tf.expand_dims(input_tensor, axis=0)

    logits = model(input_tensor, training=False)

    np.save(
        f"/content/{name}_{model.cfg.eval.isize}_original_logits.npy", 
        logits.numpy()
    )    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  100M  100  100M    0     0  54.4M      0  0:00:01  0:00:01 --:--:-- 54.4M
Model variant: efficientnetv2-b0
Train image size: 192
Eval image size: 224

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0  48.0M      0  0:00:02  0:00:02 --:--:-- 48.0M
Model variant: efficientnetv2-b1
Train image size: 192
Eval image size: 240

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  142M  100  142M    0     0  50.6M      0  0:00:02  0:00:02 --:--:-- 50.5M
Model variant: efficientnetv2-b2
Train image size: 208
Eval image size: 260

  % Total    % Received % Xferd  Average Speed   Time    T

In [ ]:
# Run for Imagenet-1k pretrained features:
for name in MODELS:
    tf.keras.backend.clear_session()

    # Do not redownload weights if present:
    ![ -e $name ] || curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{name}.tgz | tar xzf -

    model = EffNetV2Model(name, include_top=False)

    # Initialize variables:
    _ = model(tf.ones([1, model.cfg.eval.isize, model.cfg.eval.isize, 3]), training=False)
    
    # Load weights:
    ckpt_path = os.path.join(os.getcwd(), name)
    if tf.io.gfile.isdir(ckpt_path):
        ckpt_path = tf.train.latest_checkpoint(ckpt_path)
    model.load_weights(ckpt_path)

    print(f"Model variant: {name}")
    print(f"Train image size: {model.cfg.train.isize}")
    print(f"Eval image size: {model.cfg.eval.isize}")
    print()

    image = tf.image.decode_jpeg(tf.io.read_file(image_file))
    input_tensor = preprocessing.preprocess_image(
        image,
        image_size=model.cfg.eval.isize,
        is_training=False
    )
    input_tensor = tf.expand_dims(input_tensor, axis=0)

    features = model(input_tensor, training=False)

    np.save(
        f"/content/{name}_{model.cfg.eval.isize}_original_features.npy", 
        features.numpy()
    )    

Model variant: efficientnetv2-b0
Train image size: 192
Eval image size: 224

Model variant: efficientnetv2-b1
Train image size: 192
Eval image size: 240

Model variant: efficientnetv2-b2
Train image size: 208
Eval image size: 260

Model variant: efficientnetv2-b3
Train image size: 240
Eval image size: 300

Model variant: efficientnetv2-s
Train image size: 300
Eval image size: 384

Model variant: efficientnetv2-m
Train image size: 384
Eval image size: 480

Model variant: efficientnetv2-l
Train image size: 384
Eval image size: 480



In [5]:
# Collect logits for imagenet-21k pretrained variants:
MODELS = [
    "efficientnetv2-s",
    "efficientnetv2-m",
    "efficientnetv2-l",
    "efficientnetv2-xl",
]

In [6]:
for name in MODELS:
    tf.keras.backend.clear_session()
    !curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{name}-21k-ft1k.tgz | tar xzvf -

    model = EffNetV2Model(name)
    _ = model(tf.ones([1, model.cfg.eval.isize, model.cfg.eval.isize, 3]), training=False)
    
    ckpt_path = os.path.join(os.getcwd(), f"{name}-21k-ft1k")
    if tf.io.gfile.isdir(ckpt_path):
        ckpt_path = tf.train.latest_checkpoint(ckpt_path)
    
    model.load_weights(ckpt_path)

    print(f"Model variant: {name}")
    print(f"Train image size: {model.cfg.train.isize}")
    print(f"Eval image size: {model.cfg.eval.isize}")
    print()

    image = tf.image.decode_jpeg(tf.io.read_file(image_file))
    input_tensor = preprocessing.preprocess_image(
        image,
        image_size=model.cfg.eval.isize,
        is_training=False
    )
    input_tensor = tf.expand_dims(input_tensor, axis=0)

    logits = model(input_tensor, training=False)

    np.save(
        f"/content/{name}_{model.cfg.eval.isize}_21k-ft1k_original_logits.npy", 
        logits.numpy()
    )    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0efficientnetv2-s-21k-ft1k/
efficientnetv2-s-21k-ft1k/model.meta
 34  303M   34  105M    0     0  49.9M      0  0:00:06  0:00:02  0:00:04 49.9Mefficientnetv2-s-21k-ft1k/model.index
efficientnetv2-s-21k-ft1k/checkpoint
efficientnetv2-s-21k-ft1k/model.data-00000-of-00001
100  303M  100  303M    0     0  54.4M      0  0:00:05  0:00:05 --:--:-- 56.6M
Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming 

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Model variant: efficientnetv2-s
Train image size: 300
Eval image size: 384

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0efficientnetv2-m-21k-ft1k/
  0  76efficientnetv2-m-21k-ft1k/model.meta
 43  762M   43  330M    0     0  53.9M      0  0:00:14  0:00:06  0:00:08 54.6Mefficientnetv2-m-21k-ft1k/model.index
efficientnetv2-m-21k-ft1k/checkpoint
efficientnetv2-m-21k-ft1k/model.data-00000-of-00001
100  762M  100  762M    0     0  55.5M      0  0:00:13  0:00:13 --:--:-- 54.4M
Model variant: efficientnetv2-m
Train image size: 384
Eval image size: 480

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0efficientnetv2-l-21k-

In [7]:
# Get extracted feature shape for more sanity checks regarding shape:

In [8]:
# Add     print(outputs.shape)
# to line 474 in effnetv2_models.py

In [4]:
# RESTART RUNTIME

In [1]:
%cd automl/efficientnetv2/

/content/automl/efficientnetv2


In [2]:
import os
import numpy as np
import tensorflow as tf

from effnetv2_model import EffNetV2Model
import preprocessing


In [3]:
MODELS = [
    "efficientnetv2-b0",
    "efficientnetv2-b1",
    "efficientnetv2-b2",
    "efficientnetv2-b3",
    "efficientnetv2-s",
    "efficientnetv2-m",
    "efficientnetv2-l",
    "efficientnetv2-xl",
]

In [4]:
for name in MODELS:
    tf.keras.backend.clear_session()
    

    print(f"Model variant: {name}")
    model = EffNetV2Model(name, include_top=False)
    _ = model(tf.ones([1, model.cfg.eval.isize, model.cfg.eval.isize, 3]), training=False)

Model variant: efficientnetv2-b0
(1, 7, 7, 1280)
Model variant: efficientnetv2-b1
(1, 8, 8, 1280)
Model variant: efficientnetv2-b2
(1, 9, 9, 1408)
Model variant: efficientnetv2-b3
(1, 10, 10, 1536)
Model variant: efficientnetv2-s
(1, 12, 12, 1280)
Model variant: efficientnetv2-m
(1, 15, 15, 1280)
Model variant: efficientnetv2-l
(1, 15, 15, 1280)
Model variant: efficientnetv2-xl
(1, 16, 16, 1280)


In [ ]:
%cd /content/

/content


In [ ]:
!tar czvf efficientnet-v2-outputs.tar.gz *.npy

efficientnetv2-b0_224_original_features.npy
efficientnetv2-b0_224_original_logits.npy
efficientnetv2-b1_240_original_features.npy
efficientnetv2-b1_240_original_logits.npy
efficientnetv2-b2_260_original_features.npy
efficientnetv2-b2_260_original_logits.npy
efficientnetv2-b3_300_original_features.npy
efficientnetv2-b3_300_original_logits.npy
efficientnetv2-l_480_21k-ft1k_original_features.npy
efficientnetv2-l_480_21k-ft1k_original_logits.npy
efficientnetv2-l_480_original_features.npy
efficientnetv2-l_480_original_logits.npy
efficientnetv2-m_480_21k-ft1k_original_features.npy
efficientnetv2-m_480_21k-ft1k_original_logits.npy
efficientnetv2-m_480_original_features.npy
efficientnetv2-m_480_original_logits.npy
efficientnetv2-s_384_21k-ft1k_original_features.npy
efficientnetv2-s_384_21k-ft1k_original_logits.npy
efficientnetv2-s_384_original_features.npy
efficientnetv2-s_384_original_logits.npy


In [ ]:
from google.colab import files

In [ ]:
files.download("efficientnet-v2-outputs.tar.gz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>